In [3]:
!pip install mne

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 67.7 MB/s eta 0:00:00


In [9]:
!pip install mne --upgrade


In [16]:
!pip install pingouin


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.4/204.4 kB 7.8 MB/s eta 0:00:00


In [17]:
import numpy as np
import pandas as pd
import mne
from scipy.stats import ttest_ind
import pingouin as pg
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
# Charger les données MEG ou EEG au format BIDS
# Assurez-vous d'avoir le bon chemin vers vos données BIDS
bids_path = Path("/path/to/your/data")  # Remplacez par votre chemin
raw_data = mne.io.read_raw_bids(bids_path)

# Sélectionner uniquement les canaux EEG (si présents)
raw_data.pick_types(eeg=True)

# 1. **Calcul de la densité spectrale de puissance (PSD)**

In [ ]:
# Paramètres de calcul de la PSD
fmin, fmax = 1, 40  # Plage de fréquence à analyser
n_per_seg = 256  # Taille de la fenêtre pour la méthode Welch

# Calcul de la PSD avec MNE
freqs, psd = psd_welch(raw_data, fmin=fmin, fmax=fmax, n_per_seg=n_per_seg)

# Vous pouvez aussi appliquer une normalisation ou un lissage à la PSD si nécessaire
psd_mean = psd.mean(axis=-1)  # Moyenne sur toutes les fenêtres temporelles


# 2. **Test Statistique**

In [ ]:
# Supposons que vous ayez deux conditions (avant/après un stimulus)
psd_condition1 = psd_mean[:, 0]  # Moyenne PSD pour la condition 1
psd_condition2 = psd_mean[:, 1]  # Moyenne PSD pour la condition 2

# Test t de Student pour comparer les deux conditions
t_stat, p_value = ttest_ind(psd_condition1, psd_condition2)
print(f"T-test : t = {t_stat}, p = {p_value}")

# 3. **Calcul de la cohérence inter-trial (ITC)**

In [ ]:
# Segmenter les données en essais (epochs) si nécessaire
# On suppose que vous avez déjà défini les événements
events = mne.find_events(raw_data)

# Créer des epochs à partir des événements
epochs = mne.Epochs(raw_data, events, event_id=None, tmin=-0.5, tmax=1.0, baseline=(None, 0))

# Calculer l'ITC (Inter-Trial Coherence)
itc = tfr_multitaper(epochs, freqs=freqs, n_cycles=7, time_bandwidth=2.0, return_itc=True).itc

# Afficher l'ITC pour un canal (par exemple, le premier canal EEG)
print(itc[0, :, :])

# 4. **Optimisation de la Précision (Classification)**

In [ ]:
# Extraire les features (PSD et ITC)
# Ici, on suppose que vous avez des features sous forme de tableaux numpy (chaque colonne étant une feature)
X = np.vstack([psd_mean.T, itc.mean(axis=-1).T]).T  # Concaténer les features PSD et ITC
y = ['condition1', 'condition2', 'condition3']  # Remplacez par vos labels de conditions

# Diviser les données en jeu d'entraînement et test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
# Créer et entraîner un classifieur SVM (Support Vector Machine)
clf = SVC()
clf.fit(X_train, y_train)

In [ ]:
# Prédire sur le jeu de test et calculer la précision
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")